# Search - path planning

## First search
1. Start with a grid with a start position S and goal position G.
2. Expand from S into all directions that are available (not blocked off).
3. Keep track of the path length for each cell (g-value).
4. Find the cell with the current shortest path length and expand from there.

In [1]:
import numpy as np

In [7]:
# ----------
#
# Define a function, search() that returns a list
# in the form of [optimal path length, row, col]. For
# the grid shown below, your function should output
# [11, 4, 5].
#
# If there is no valid path from the start point
# to the goal, your function should return the string
# 'fail'
# ----------

# Grid format:
#   0 = Navigable space
#   1 = Occupied space


grid = [[0, 0, 1, 0, 0, 0],
        [0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 1, 0],
        [0, 0, 1, 1, 1, 0],
        [0, 0, 0, 0, 1, 0]]
init = [0, 0]
goal = [len(grid)-1, len(grid[0])-1]
cost = 1

delta = [[-1, 0], # go up
        [ 0,-1], # go left
        [ 1, 0], # go down
        [ 0, 1]] # go right

delta_name = ['^', '<', 'v', '>']

In [12]:
def search(grid, init, goal, cost, path_length=None, verbose=False):
    if init == goal:
        return ([int(path_length[goal[0], goal[1]]), init[0], init[1]])

    if not isinstance(grid, np.ndarray):
        grid = np.array(grid)
    else:
        grid = grid.copy()

    delta = np.array([
        [-1, 0], # go up
        [ 0,-1], # go left
        [ 1, 0], # go down
        [ 0, 1] # go right
        ])

    # path_length: initialise as nan
    if path_length is None:
        path_length = np.zeros(grid.shape)
        path_length[:, :] = np.nan
        # add initial cell with path length 0
        path_length[init[0], init[1]] = 0

    # mark off the current cell
    grid[init[0], init[1]] = 1

    # check whether neighbouring cells are unmarked (untraveled)
    for d in delta:
        index = init - d
        try:
            # ignore negative indices
            if any(index < 0):
                continue

            cell = grid[index[0], index[1]]
        except(IndexError):
            # ignore out of bounds indices
            continue

        # if cell is unmarked, update the path cost
        if cell == 0:
            path_length[index[0], index[1]] = path_length[init[0], init[1]] + cost

    try:
        # find the next cell: unmarked cell with lowest cost
        unmarked = np.nonzero(grid == 0)
        lowest_cost = np.nonzero(path_length[unmarked] == np.nanmin(path_length[unmarked]))
        # next cell
        init = [unmarked[0][lowest_cost][0], unmarked[1][lowest_cost][0]]
    except:
        return "fail"

    if verbose:
        print(f"GRID:\n{grid}")
        print(f"PATH COST:\n{path_length}")

    # run the next iteration with the new cell
    return search(grid, init, goal, cost, path_length=path_length, verbose=verbose)


In [13]:
out = search(grid, init, goal, cost, verbose=True)

GRID:
[[1 0 1 0 0 0]
 [0 0 1 0 0 0]
 [0 0 0 0 1 0]
 [0 0 1 1 1 0]
 [0 0 0 0 1 0]]
PATH COST:
[[ 0.  1. nan nan nan nan]
 [ 1. nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]]
GRID:
[[1 1 1 0 0 0]
 [0 0 1 0 0 0]
 [0 0 0 0 1 0]
 [0 0 1 1 1 0]
 [0 0 0 0 1 0]]
PATH COST:
[[ 0.  1. nan nan nan nan]
 [ 1.  2. nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]]
GRID:
[[1 1 1 0 0 0]
 [1 0 1 0 0 0]
 [0 0 0 0 1 0]
 [0 0 1 1 1 0]
 [0 0 0 0 1 0]]
PATH COST:
[[ 0.  1. nan nan nan nan]
 [ 1.  2. nan nan nan nan]
 [ 2. nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]]
GRID:
[[1 1 1 0 0 0]
 [1 1 1 0 0 0]
 [0 0 0 0 1 0]
 [0 0 1 1 1 0]
 [0 0 0 0 1 0]]
PATH COST:
[[ 0.  1. nan nan nan nan]
 [ 1.  2. nan nan nan nan]
 [ 2.  3. nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]]
GRID:
[[1 1 1 0 0 0]
 [1 1 1 0 0 0]
 [1 0 0 0 1 0]
 [0 0 1 1 1 0]
 [0 0 0 0 1 0]]
PA

In [10]:
out

[11, 4, 5]

## Expansion grid